<h1> Task 1, Assessment 1 : FIT 5196 </h1>

**Student Name : Potluri Karthik Venkat Chowdary**

**Student ID : 32157738**

**Date : 13/04/2021**

<h2> Introduction </h2>

In this task we will be extracting text from semi-structured, this is the first step of analyzing any textual data. In real world textual data is always obtained in formats that are hard to extract from and perform any operations on. We can use various techniques to extract the requried information from them. In this project we will be mainly using regular expressions. 

<h4> a. Data Description </h4>

- We have a collection of text files each contains information about several news articles including the article itself.
- News articles contain information from across the globe and might also include articles that are not of English language.

**Input Files :**

- We have around 15 text files each containing information about several news articles.
- Our object of interest among the several other data points would be the uuid, author, Published and text.

**Possible Roadblocks :**

- Extracting the exact information that is neccesary.
- Excluding Articles that are not in English.
- Converting special symbols.
- Avoiding commas while writing to CSV


<h4> b. Libraries </h4>

For this project we will be keeping our use of libraries to minimum. We will be using only three libraries, which are as follows: 

- **re :** We will be using re for operating with regular expressions.
- **os :** os will be used to read and write information and manipulate files.
- **langid :** We will be using Langid to classify the language of articles to exclude articles that are not in English.

In [3]:
#Importing Packages
import re
import os
import langid

<h2> Parsing files and Extracting information </h2>

<h4> a. Parsing files </h4>

We will be using the package OS while parsing the files and reading in the information. We will be going through all the files in the folder "32157738" and saving their names into a list called files_shake.

In [4]:
import os
files_shake = []
for root, dirs, files in os.walk("./32157738/"):
    for filename in files:
        files_shake.append("./32157738/"+filename)

<h4> b. Extracting information </h4>

Two extract the data using regular expression it is very important to understand the form in which data is currently in the text file.Each text file have around a 100 news articles in them along with a lot of details about these news articles. Regular expression searches patterns and strings to return the required pattern or string. However a 8-bit string and an unicode cannot be matched using regular expressions (Re — Regular Expression Operations — Python 3.9.4 Documentation, n.d.). Let us understand the existing data better, before proceding to create our regular expressions.

If we look at the text files we can observe that they look something like this

$$   
.... "uuid": \textit{"uuid"}, \hspace{3mm} "author": \textit{"author"}, \hspace{3mm} "published": \textit{"published"}, \hspace{3mm} "text": \textit{"text"}, \hspace{3mm} "country": .... 
$$

Let us first concentrate on getting uuid from this, 
- To obtain uuid we know that we have to search for text after ___"uuid": "___ 
- And uuid ends before author starts, so uuid is the value that is between ___"uuid": "___ and ___" "author"___
- There is a problem here, as there are multiple ___"uuid": "___ and ___" "author"___ we need to specify in the regular expression to extract the information between to succesive ___"uuid": "___ and ___" "author"___ . We can do this by using a ? symbol. 
- while representing symbols such as ' _"_ ', ' _:_ ' which are also present in regular expressions we need to put a back slash before using them.
- Another important step while using the regular expression is representing the white space, to represent the white space we will be using a _'/s'_ .
- We will be going through each text file in _files_shake_ and use ___re.findall___ to get every possible occurance that satisfies our conditions.

Similarly by using the above process we can extract other attributes (author, published and text) from the text file.

- I performed a small check here to compare the number of uuid, author, published and text. Since they all should be equal. 
- If they are all equal then we cann add them to the list of uuid (_uuid_list_), author (_author_list_), published (_published_list_), and (_text_list_).



In [5]:
uuid_list = []
author_list = []
published_list = []
text_list = []
for file_name in files_shake:
    f = open(file_name, "r", encoding = "utf8")
    temp_file = f.read()
    uuid_list_temp = re.findall('\"uuid\"\:\s\"(.*?)\"\,\s\"author\"',temp_file,flags=re.DOTALL)
    author_list_temp = re.findall('\"author\"\:\s\"(.*?)\"\,\s\"published\"',temp_file,flags=re.DOTALL)
    published_list_temp = re.findall('\"published\"\:\s\"(.*?)\"\,\s\"text\"',temp_file,flags=re.DOTALL)
    text_list_temp = re.findall('\"text\"\:\s\"(.*?)\"\,\s\"country\"',temp_file,flags=re.DOTALL)
    
    if(len(uuid_list_temp)==len(author_list_temp)==len(published_list_temp)==len(text_list_temp)):
        uuid_list.extend(uuid_list_temp)
        author_list.extend(author_list_temp)
        published_list.extend(published_list_temp)
        text_list.extend(text_list_temp)
    else:
        print("something is wrong in the file "+file_name)


<h4> c. Including English only articles </h4>

- Among the articles we have now, some of the articles are not in English, we are not interested in these articles. We will be removing them.
- To check whether an article is in English or not we will be using classify function of langid. This gives a result like this : ('en' , 349030130 ) if the language is english. If it is not english 'en' would be different 'ru' for russian, 'hi' for hindi etc.,
- We will be checking each article if it is english are not and if it is english we will be adding it and corresponding uuid, author and published to their own lists.

In [6]:
uuid_list_2 = []
author_list_2 = []
published_list_2 = []
text_list_2 = []
list_length = len(uuid_list)
for i in range(list_length):
    if langid.classify(text_list[i])[0] == 'en':
        uuid_list_2.append(uuid_list[i])
        text_list_2.append(text_list[i].replace('\\n',''))
        author_list_2.append(author_list[i])
        published_list_2.append(published_list[i])
    else:
        continue
       


We have removed all the articles that are not in English. Our next step would be to convert special characters. 

<h4> d. Special characters </h4>

- If we look at the data now we can observe that there are certain characters which or still in their unicode (\\u1234) format.
- Unicode is an IT standard used for encoding text expressed in most of the written langauges _(Wikipedia contributors, n.d.)_.
- These unicode can be converted into their original symbols by treating them as surrogate pairs _(Surrogate Characters | IOS Internationalization: Characters and Encoding | InformIT, 2015)_.
- We will be encoding and decoding the files using the surrogate pass. we will be doing this process until all the articles are devoid of unicode symbols.

Stop = 1
while stop == 0 :
    list_length_2 = len(uuid_list_2)
    Stop = 0
    for j in range(list_length_2):
        if re.search(r'(\\u){1}',text_list_2[j]) is not None:
            text_list_2[j] = "\""+text_list_2[j]+"\""
            text_list_2[j] = eval(text_list_2[j]).encode('utf-16', 'surrogatepass').decode('utf-16')
            if re.search(r'(\\u){1}',text_list_2[j]) is not None:
                stop = 1
        else:
            text_list_2[j] = text_list_2[j].encode('utf-16', 'surrogatepass').decode('utf-16')

<h2> Writing to XML and CSV </h2>

Now that we have all the english articles in place with no special characters or symbols. We can write them to XML and CSV with small necessary changes.

<h4> a. Writing to XML </h4>

XML has some special characters in it that cannot exist in the text, If they do it will obstruct the XML format.

- To avoid these we will be replacing them with appropriate symbols from XML. 
- We wiill be writing the existing items from each of the lists one by one into the XML file as below.

In [7]:
with open('32157738.xml','w',encoding='utf-8') as f:
    f.write('<?xml version="1.0" encoding="UTF-8"?>')
    f.write('<data>\n')
    for i in range(len(uuid_list_2)):
        f.write('<item>\n')
        f.write('<uuid>'+uuid_list_2[i].replace("\\n", " ").replace("\\", "&#92;").replace("<", "&#lt;").replace(">", "&#62;").replace("\'", "&apos;").replace("&", "&amp;").replace("\"", "&quot;")+'</uuid>\n')
        f.write('<author>'+author_list_2[i].replace("\\n", " ").replace("\\", "&#92;").replace("<", "&#lt;").replace(">", "&#62;").replace("\'", "&apos;").replace("&", "&amp;").replace("\"", "&quot;")+'</author>\n')
        f.write('<published>'+published_list_2[i].replace("\\n", " ").replace("\\", "&#92;").replace("<", "&#lt;").replace(">", "&#62;").replace("\'", "&apos;").replace("&", "&amp;").replace("\"", "&quot;")+'</published>\n')
        f.write('<text>'+text_list_2[i].replace("\\n", " ").replace("\\", "&#92;").replace("<", "&#lt;").replace(">", "&#62;").replace("\'", "&apos;").replace("&", "&amp;").replace("\"", "&quot;")+'</text>\n')
        f.write('</item>\n')
    f.write('</data>\n')


<h4>b. Writing to CSV </h4>

While writing to CSV the characters that would prove as issue would be  Comma(,), Double quotes ("), Line break("\n") so as a blanket measure we will be replacing them with multiple quotes so we would not stumble into problems with them.

In [8]:
with open('32157738.csv','w',encoding='utf-8') as h:
    h.write("uuid,author,published,text\n")
    for i in range(len(uuid_list_2)):
        h.write('"'+uuid_list_2[i].replace("\n","").replace('"', '""')+'"'+","+'"'+author_list_2[i].replace("\n"," ").replace('"', '""')+'"'+","+'"'+published_list_2[i].replace("\n","").replace('"', '""')+'"'+","+'"'+text_list_2[i].replace("\'","'").replace("\n","").replace('"', '""')+'"'+"\n")

<h2> References </h2>

- re — Regular expression operations — Python 3.9.4 documentation. (n.d.). Python.Org. https://docs.python.org/3/library/re.html
- Wikipedia contributors. (n.d.). Unicode. Wikipedia. https://en.wikipedia.org/wiki/Unicode
- Surrogate Characters | iOS Internationalization: Characters and Encoding | InformIT. (2015, January 21). Informit. https://www.informit.com/articles/article.aspx?p=2274038&seqNum=10